In [1]:
include("../src/Julia.jl")

parseMatlabData (generic function with 1 method)

In [5]:
data = []

ipmFolder = "../../graphs/ipm_grid3_8000_all/"
allGraphs = readdir(ipmFolder)

ipmIter = 0
data = [Array{Float64,1}(0) for i in 1:5] # a column for each precision
while true
    try
        ipmIter = ipmIter + 1
        defaultName = ipmFolder * "ipm_grid3_8000_$(ipmIter)/" * "_log_julia_tree1_default.txt"
        defaultNorm = parseData(defaultName, only2=true)

        ind = 0
        for precision in [64, 128, 256, 512, 1024]
            logName = ipmFolder * "ipm_grid3_8000_$(ipmIter)/" * "_log_julia_tree1_$(precision).txt"
            norm = parseData(logName, only2=true)

            convIter=1
            for i in 1:100
                if norm[i] < 0.1
                    convIter = i
                    break
                end
            end

            ind = ind + 1
            push!(data[ind], norm[convIter] / defaultNorm[convIter])
        end

#             println("finished step ", ipmIter)
    catch
        # we aggregated all the graphs
        break
    end
end

for i in 1:5
    plot(log(data[i]), label="$(2^(i+5)) / Default", linewidth=2.5)
end
legend(fontsize=7)
ylabel("[High Precision / Default] at Convergence")
xlabel("IPM iteration")

savefig(ipmFolder * "convergencePlot.png")

PyPlot.clf()